In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.memory import ConversationSummaryBufferMemory
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.schema.runnable import RunnablePassthrough

llm = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY,
    model_name="gpt-3.5-turbo",
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=150,
    return_messages=True,
    memory_key="history",
)

examples = [
    {"movie": "탑건", "answer": "🛩️👨‍✈️🔥"},
    {"movie": "대부", "answer": "👨‍👨‍👦🔫🍝"},
    {"movie": "인셉션", "answer": "🔱🎆🛫"},
    {"movie": "아바타", "answer": "💦🪂🏝"},
]
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "Express {movie} as three emojis."),
        ("ai", "{answer}"),
    ]
)
prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are best movie lover that can express it as three emojis."),
        MessagesPlaceholder(variable_name="history"),
        prompt,
        ("human", "{movie}"),
    ]
)

def load_memory(_):
    return memory.load_memory_variables({})["history"]

chain = RunnablePassthrough.assign(history=load_memory) | final_prompt | llm

def invoke_chain(movie):
    result = chain.invoke({"movie": movie})
    memory.save_context(
        {"input": movie},
        {"output": result.content},
    )
    print(result)